In [1]:
import torch, os
import torch.nn.functional as F
import lightning as pl
from einops import rearrange
import matplotlib.pyplot as plt
from tqdm import tqdm
from einops import rearrange

from torchvision.utils import save_image, make_grid
from torchvision import transforms
from torchvision.datasets import MNIST

from modules.trainers.DDPM import DDPM

In [2]:
model = DDPM.load_from_checkpoint('/home/exouser/genmod/genmod/vbczy3ao/checkpoints/last.ckpt')
model.eval()

DDPM(
  (nn_model): ContextUnet(
    (init_conv): ResidualConvBlock(
      (conv1): Sequential(
        (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): GELU(approximate='none')
      )
      (conv2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): GELU(approximate='none')
      )
    )
    (down1): UnetDown(
      (model): Sequential(
        (0): ResidualConvBlock(
          (conv1): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (2): GELU(approximate='none')
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, k

In [4]:
device = "cuda:0"

torch.cuda.empty_cache()

model = model.to(device)

gen_imgs = []
for _ in range(9):
    x_i = torch.randn((1,1,28,28)).to(device)  # x_T ~ N(0, 1), sample initial noise
    x_0 = model.sample_loop(x_i)

    gen_imgs.append(x_0.squeeze(0).detach().cpu())
    del x_0
    
gen_imgs = rearrange(gen_imgs, 'b c h w -> b c h w')
# create grid
grid = make_grid(gen_imgs*-1 + 1, nrow=3, padding=2)

# plot grid
plt.imshow(grid.permute(1, 2, 0))
plt.axis('off')
plt.show()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 40.00 GiB total capacity; 7.45 GiB already allocated; 109.94 MiB free; 7.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF